read .mtx form data from website: www.math.nist.gov/MatrixMarket/data


In [66]:
using DelimitedFiles, SparseArrays, LinearAlgebra
dir  = "data/well1850.mtx/"
ln = "well1850.mtx"

A1=readdlm(string(dir,strip(ln));header=false,skipstart=2) # First, comment out the following code and use this code to check
# the structure of data.
S1=Int.(A1[:,1])
S2=Int.(A1[:,2])
# make S3.>0
A1[:,3] .-= minimum(A1[:,3])
S3=A1[:,3]


A=sparse( Int64.(A1[:,1]), Int64.(A1[:,2]), convert(Array{Float64},A1[:,3]))

b=ones(maximum(Int64.(A1[:,1])))./10

1850-element Array{Float64,1}:
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 ⋮
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1

Read data from website: www.csie.ntu.edu.tw

In [1]:
using DelimitedFiles, SparseArrays, LinearAlgebra, Parsers
dir  = "data/real-sim/"
ln = "real_sim"

b=[0]
S1=[0]
S2=[0]
S3=[0.0]

open(string(dir,strip(ln))) do f
    for (i, line) in enumerate(eachline(f))
        if(line[1]=='1')
            append!(b,1)
        else
            append!(b,-1)
        end
        s=line[2]
        ind1=findfirst(isequal(':'),line)
        ind2=findfirst(isequal(' '),line)
        if(ind1==nothing) # This step is to prevent triviality happenning, that is there is an empty raw in A matrix.
            continue
        end
        ind3=findnext(isequal(' '),line,ind1)
        while ind1!=nothing
            append!(S1,i)
            append!(S2,Parsers.parse(Int,line[(ind2+1):(ind1-1)]))
            append!(S3,Parsers.parse(Float64,line[(ind1+1):(ind3-1)]))
#             print(line[(ind2+1):(ind1-1)],"\n")
            ind2=ind3
            ind1=findnext(isequal(':'),line,ind2)
            ind3=findnext(isequal(' '),line,ind3+1) 
        end
        #print("\n$i, $ind1, $ind2, $ind3\n")
    end
end
b=b[2:end]
S1=S1[2:end]
S2=S2[2:end]
S3=S3[2:end]

# print(S1)
# print(S2)
# print(S3)
# print(length(b),",")
# print(length(S1),",")
# print(length(S2),",")
# print(length(S3))

b .+= 2 # make the b vector positive.
A=sparse(Int64.(S1), Int64.(S2), convert(Array{Float64},S3))

72309×20958 SparseMatrixCSC{Float64,Int64} with 3709083 stored entries:
  [6189 ,     1]  =  0.04629
  [7166 ,     1]  =  0.0353996
  [7187 ,     1]  =  0.0265096
  [22243,     1]  =  0.115242
  [22250,     1]  =  0.0197798
  [22259,     1]  =  0.0680985
  [22260,     1]  =  0.051798
  [22263,     1]  =  0.042465
  [22270,     1]  =  0.215061
  [22272,     1]  =  0.131526
  [22274,     1]  =  0.0390799
  [22281,     1]  =  0.0850954
  ⋮
  [15689, 20958]  =  0.0660176
  [20325, 20958]  =  0.120234
  [21041, 20958]  =  0.144848
  [21045, 20958]  =  0.100601
  [23382, 20958]  =  0.095951
  [33345, 20958]  =  0.13035
  [33507, 20958]  =  0.121403
  [35824, 20958]  =  0.108362
  [51738, 20958]  =  0.495118
  [51770, 20958]  =  0.136784
  [51791, 20958]  =  0.0977064
  [62161, 20958]  =  0.0996344
  [62286, 20958]  =  0.0783678

In [5]:
dir  = "data/wiki/"
ln = "wiki10_31k_tfidf_train.svm"
A=readdlm(string(dir,strip(ln));header=false,skipstart=0) # multi-label case. Need to define b vector. Can similarly use the previous block to extract A.

14146×4769 Array{Any,2}:
 "2980,3177,9026,9053,12256,12258,14385,14548,21806,23227,24101,24485,25398,28816,28825,28828,28835,28838,28840,30071,30134,30708,30709"                                  …  ""  ""  ""  ""  ""  ""  ""  ""  ""
 "4188,14485,14932,20047,20085,20165,24290,26519,27814,29121,30134,30177"                                                                                                    ""  ""  ""  ""  ""  ""  ""  ""  ""
 "1416,4633,9483,9580,12288,12290,14123,21775,24290,25348,25349,28376,28828,30071,30134,30546"                                                                               ""  ""  ""  ""  ""  ""  ""  ""  ""
 "1258,1633,3610,3623,4873,11855,12750,17148,17901,22146,22225,22392,23546,24073,24733,27991,28353,30134,30456,30459"                                                        ""  ""  ""  ""  ""  ""  ""  ""  ""
 "2303,2304,3737,3775,4688,5768,6685,6686,7076,7353,7432,10169,10220,15065,18635,19659,22013,24998,25004,25168,25679,29833,29902,29917,29918"  

Test For NNLS

In [77]:
using LinearAlgebra, NonNegLeastSquares

function alg_ours_with_restart(C, b::Matrix{Float64}, ϵ::Float64 )
    extra_term_nnls = 0.5*norm(b)^2
    m, n = size(C)
    #number of times we restart
    K = 50 #actually doesn't matter, since the restart_metric becomes really tiny pretty quickly 
    col_norm = norm.(eachcol(C))
    inv_col_norm_square = 1.0 ./(col_norm.^2)
    idx_seq = 1:n

    x0 = zeros(n)
    y0 = zeros(m)
    z0 = zeros(m)

    gamma = 30 #chosen after experiments on synthetic data
    obj = 0 
    
    init_time = time()
    for i=1:K

        xktilde, yktilde, zktilde, new_metric, obj = alg_our_core(C, x0, y0, z0, m, n, inv_col_norm_square, idx_seq, ϵ, gamma, init_time, extra_term_nnls)

        if (new_metric < ϵ)
            break
        end
        x0[:] = xktilde[:]
        y0[:] = yktilde[:]
        z0[:] = zktilde[:]

        gamma/= 2 # chosen after experiments on synthetic data 
    end
    return obj
end

function alg_our_core(C, x0, y0, z0, m, n, inv_col_norm_square, idx_seq, ϵ, gamma, init_time, extra_term_nnls)
        # reset all the scaling factors
        previous_A = 1.0/n
        previous_a = previous_A #a_1, A_1
        a = 1.0/(n*n) # a_2
        A = (n+1.0)/(n * n) # A_2


        # compute x1 using the input x0
        # we redefined phio(x) = 1/2 * ||x-x0||_A^2, hence updating x requires x0
        # the step p(j)+=1/||A:j||^2 implicitly assumes ybar_0 = 0
        # To allow for ybar_0 \neq 0, we change p(j) a bit

        ybar = copy(y0)
        j = rand(idx_seq)
        Aty0m = 1 - dot(ybar, C[:, j]) #                                     dot(̄ȳ, C[:, j])

        p = copy(x0)
        x = copy(x0)
        p[j] += inv_col_norm_square[j]*Aty0m
        x[j] = min(inv_col_norm_square[j], max(0, p[j])) #x and x0 differ only at j

        # compute y1
        # note that y0^(R) and y1^(R) are independent of each othre
        # y1^(R) = Ax1^(R) = Ax0^(R) + A*(x1^R - x0^R) = z0^R + A*(x1^R - x0^R)
        # y0^R may be chosen to be either ytildeK or 0 (our analysis uses 0)
        # Further note that if y0^R = ytildeK, then we must ALSO choose ybar_0 = ytildeK, and
        # this changes how x is init.
        previous_y = copy(y0)
        z = copy(z0)
        z += C[:, j] * (x[j] - x0[j]) # z_1 = A x_1 = A (x_0 + (x_1 - x_0))
        y = copy(z) # y_1 = A xtilde1 = A x_1 = z_1

        # compute ȳ, ỹ (because we need to return it), and some auxiliary variables
        ybar[:] = y[:] + previous_a/a * (y[:] - previous_y[:]) #ybar_1
        s = zeros(n) # need this so that xtildek = xk + sk/Ak; s_1 = 0 (see Chaobing's lemma for why this is needed)
        ỹ = copy(y) # ytildek = convex comb of yi's, so ytilde1 = y1

        # restart value init; -1^{\top}x+0.5\|Ax\|^{2}+.5*\|y\|^{2}+\frac{1}{2\epsilon}\|(-A^{\top}y+1)^{+}\|^{2}
        restart_coeff = 5000
        Atym = -C'*y0 .+ 1
        truncated_Atym = ((Atym) .> 0).*Atym
        restart_val_prev = -sum(x0)+ 0.5* norm(z0)^2 +0.5*norm(y0)^2 + restart_coeff*norm(truncated_Atym)^2
        restart_val_curr = restart_val_prev

        # inits for restart
        iter_count = 0
        Flag = true
        Ax0 = zeros(m)
        new_metric = 0 
        obj = 0 
    
        while (Flag)

            # updates related to x
            j = rand(idx_seq)
            p[j] += - n * inv_col_norm_square[j] * a * (sum(C[:,j] .* ybar) - 1)
            prev_xj = x[j]
            x[j] = min(inv_col_norm_square[j], max(0, p[j]))
            # update s so that we may return xtildek at only O(1) cost
            s[j] += ((n-1) * a -  previous_A) * (x[j] - prev_xj)

            # updates related to y
            previous_y[:] = y[:]
            z[:] += C[:, j] * (x[j] - prev_xj)
            y[:] = previous_A/A * y[:] + a/A * z[:] + (n-1) * a/A * (x[j] - prev_xj) * C[:,j]
            # need to update ytilde each time because that's what we want to return,
            # and we aren't saving all the yi's.
            ỹ[:] = previous_A/A * ỹ[:] + a/A * y[:]

            # update scaling factors
            previous_a, previous_A = a, A
            a = min(n * a/(n-1), sqrt(A)/(2*n))
            A += a

            # update ȳ (note that ȳ_k depends on a_k and a_{k+1})
            ybar[:] = y[:] + previous_a/a * (y[:] - previous_y[:])

            # restart stuff
            iter_count+=1
            # Since we are computing the restart condition without any optimizations,
            # and the restart condition likely involves (expensive) matrix-vector products,
            # we check it only after a certain number of iters have passed.

            if (iter_count % ceil(n*gamma) ==0)
                # compute the restart metric 
                # note that we DO use the restart metric to terminate the OUTER ALG, 
                # , even though we are doing fixed restarts in the INNER ALG 
                Atym = -C'*ỹ .+ 1
                truncated_Atym = ((Atym) .> 0).*Atym
                sumx0 = sum(x + (1.0/previous_A) * s)
                Ax0 = C*(x + (1.0/previous_A) * s)
                new_metric = norm(truncated_Atym)^2

                # For now, we are doing fixed restarts.
                obj = extra_term_nnls-sumx0+ 0.5* norm(Ax0)^2
                print("\n\n Obj = ", obj, ", time since init = ", time() - init_time, ", new metric = ", new_metric)
                print("\n")
                Flag = false
                # end
            end
        end
        return x + (1.0/previous_A) * s, ỹ, Ax0, new_metric, obj
end

alg_our_core (generic function with 1 method)

In [67]:
function alg_lawsonhanson(A, b)
    xnnls = nonneg_lsq(A,b;alg=:nnls)  # NNLS

    nnls_optval = 0.5*norm(A*xnnls - b)^2
    
    print("\n\n nnls package value is ", nnls_optval, ", and time is ")
end

function remove_col1(A,b)#Chenghui has an idea to optimize this for speed ("filter")
    s=A'*b # n*1
    B=A[:,vec(s.>0)] # m*b matrix where b is smaller than n
    s=s[vec(s.>0)] # s is b*1 in dimensions
    return B./s'
end

epsilon = 0.001

# Write a sparse version remove_col1(A,b)
function remove_col1_sparse(A,S1,S2,S3,b)# This only holds when b.>0
    s=A'*b # n*1
#     B=A[:,vec(s.>0)] # m*b matrix where b is smaller than n
#     s=s[vec(s.>0)] # s is b*1 in dimensions
    for i in 1:length(S1)
        S3[i] = S3[i]/s[S2[i]]
    end
    return sparse(Int64.(S1), Int64.(S2), convert(Array{Float64},S3))
end

# Sparse Test

# A_init = max.(0, sprandn(1000,100,0.5))
# b1 = ones(1000)

# A= remove_col1(A_init,b1)
# A= sparse(A)


A = remove_col1_sparse(A,S1, S2, S3, b)
b=Float64.(b)

#  @time begin
#  our_result = alg_ours_with_restart(A, vcat(b'), epsilon)
#  end

@time begin
    xnnls = nonneg_lsq(A,b;alg=:nnls)  # NNLS
    nnls_optval = 0.5*norm(A*xnnls - b)^2
    print("\n\n nnls package value is ", nnls_optval, ", and time is ")
end



 nnls package value is 9.852741742974221e-28, and time is   1.468485 seconds (47 allocations: 10.130 MiB)


In [80]:
 @time begin
     our_result = alg_ours_with_restart(A, vcat(b'),epsilon)
     print("\n\n Our result with restart is ", our_result, ", and time is ")
 end
#remove_col1_sparse(A,S1, S2, S3, b)



 Obj = 0.06416329572699198, time since init = 5.180999994277954, new metric = 6.992740190115554


 Obj = 0.05556134400457502, time since init = 7.900000095367432, new metric = 0.25541727205554676


 Obj = 0.05505137213330791, time since init = 9.312999963760376, new metric = 0.20609047604819394


 Obj = 0.05497004582145415, time since init = 10.044000148773193, new metric = 0.2024810365001493


 Obj = 0.054936933025510726, time since init = 10.411999940872192, new metric = 0.20188464565447461


 Obj = 0.05491877340331186, time since init = 10.594000101089478, new metric = 0.20146358417347532


 Obj = 0.054912261809191065, time since init = 10.686000108718872, new metric = 0.20126774600163339


 Obj = 0.05489693838661225, time since init = 10.739000082015991, new metric = 0.20265528532398158


 Obj = 0.054882486960169174, time since init = 10.761000156402588, new metric = 0.20170961506319424


 Obj = 0.054869963823735546, time since init = 10.776000022888184, new metric = 0.2023112066

In [78]:
@time begin
    xnnls = nonneg_lsq(Array(A),vcat(b);alg=:nnls)  # NNLS
    nnls_optval = 0.5*norm(A*xnnls - vcat(b))^2
    print("\n\n nnls package value is ", nnls_optval, ", and time is ")
end



 nnls package value is 9.852741742974221e-28, and time is   1.368867 seconds (53 allocations: 20.209 MiB)


In [83]:
 @time begin
     our_result = alg_ours_with_restart(Array(A), vcat(b'),0.00001)
     print("\n\n Our result with restart is ", our_result, ", and time is ")
 end



 Obj = 0.009792827611532218, time since init = 2.255999803543091, new metric = 0.021822663187869278


 Obj = 0.008668527035482043, time since init = 3.2749998569488525, new metric = 0.000870951195201559


 Obj = 0.008516983731849948, time since init = 3.7909998893737793, new metric = 0.000670784431547512


 Obj = 0.008506446064739492, time since init = 4.055999994277954, new metric = 0.0007087274331989378


 Obj = 0.008505232972641608, time since init = 4.194999933242798, new metric = 0.0006878855643249366


 Obj = 0.00850479077795363, time since init = 4.271999835968018, new metric = 0.0006940171810347759


 Obj = 0.00850455984920373, time since init = 4.310999870300293, new metric = 0.0006904105733971016


 Obj = 0.008504503514735973, time since init = 4.3379998207092285, new metric = 0.0006891950823841931


 Obj = 0.008504294209137342, time since init = 4.361999988555908, new metric = 0.0006920385667674062


 Obj = 0.008504241367186083, time since init = 4.375, new metric = 0.0006